<a href="https://colab.research.google.com/github/tafakuri/amazon-sagemaker-fine-tune-and-deploy-wav2vec2-huggingface/blob/main/Experiment_speech_enhancement_impact_on_labeling_different_enhancement_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!sudo apt-get install git-lfs tree

Traceback (most recent call last):
  File "/usr/bin/add-apt-repository", line 363, in <module>
    addaptrepo = AddAptRepository()
  File "/usr/bin/add-apt-repository", line 41, in __init__
    self.distro.get_sources(self.sourceslist)
  File "/usr/local/lib/python3.10/dist-packages/aptsources/distro.py", line 91, in get_sources
    raise NoDistroTemplateException(
aptsources.distro.NoDistroTemplateException: Error: could not find a distribution template for Ubuntu/jammy
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]

In [30]:
!python3 -m pip install --upgrade pip

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [31]:
!pip install -q datasets>=2.6.1
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q librosa
!pip install -q evaluate>=0.30
!pip install -q jiwer
!pip install -q gradio
!pip install -q wandb==0.15.0
!pip install -q python-dotenv

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3

In [32]:
!pip install -qq pyannote.audio==2.1.1
!pip install -qq rich

!pip install -q torchaudio
!pip install -q speechbrain

!pip install -q webrtcvad
!pip install -q librosa
!pip install -q "soundfile>=0.12.1"

DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author

In [33]:
import dotenv
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [34]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import IPython.display as ipd

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Download dataset

In [222]:
# Select CUDA device index
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

language = "Swahili"
language_abbr = "sw"
task = "transcribe"

dataset_name = "mutisya/common_voice_11_0_sw_enhanced_sample_1"

scored_dataset_name = "mutisya/common_voice_11_0_sw_scores_sample_1"



In [36]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, use_auth_token=True)


print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2066: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'audio_vf', 'audio_sb', 'audio_sb_on_vf'],
            num_rows: 1000
        })
    })
})


In [37]:
cv_test1 = common_voice["train"]

In [38]:
#cv_test1 = shuffled_dataset.select(range(1000))

In [39]:
cv_test1

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'audio_vf', 'audio_sb', 'audio_sb_on_vf'],
        num_rows: 1000
    })
})

In [40]:
from datasets import Audio
split_dataset = cv_test1.cast_column("audio", Audio(sampling_rate=16_000)).cast_column("audio_vf", Audio(sampling_rate=16_000)).cast_column("audio_sb", Audio(sampling_rate=16_000)).cast_column("audio_sb_on_vf", Audio(sampling_rate=16_000))

In [41]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'audio_vf', 'audio_sb', 'audio_sb_on_vf'],
        num_rows: 1000
    })
})

# Select audio

In [42]:
import tempfile
import torch
import numpy as np
import soundfile as sf
import librosa

#import torchaudio

USE_CUDA = torch.cuda.is_available()
SAMPLE_RATE=16000

In [43]:
import re
#chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\ \`\(\)\*\[\](0-9)]'
chars_to_remove_regex = "[^a-zA-Z’'ũĩŨĨ\- ]"

def remove_special_characters(sentence):
    output = ""
    try:
        output = re.sub(chars_to_remove_regex, '', sentence).lower().strip()
    except:
        output = ""
    return output

In [202]:
import random

rand_int = random.randint(0, len(split_dataset["train"])-1)

labels = remove_special_characters(split_dataset["train"][rand_int]["sentence"])

print(labels)

ipd.Audio(data=split_dataset["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

kuinadhifisha meli ili iweze kuendelea kufanya kazi kwa miaka mingi zaidi


In [204]:
audio_array = split_dataset["train"][rand_int]["audio"]["array"]
audio_vf = split_dataset["train"][rand_int]["audio_vf"]["array"]
audio_sb = split_dataset["train"][rand_int]["audio_sb"]["array"]
audio_sb_on_vf = split_dataset["train"][rand_int]["audio_sb_on_vf"]["array"]


In [205]:
ipd.Audio(audio_array, autoplay=True, rate=16000)


In [206]:
ipd.Audio(audio_vf, autoplay=True, rate=16000)

In [207]:
ipd.Audio(audio_sb, autoplay=True, rate=16000)

In [208]:
ipd.Audio(audio_sb_on_vf, autoplay=True, rate=16000)

In [183]:
import librosa

filepath = '/content/embu_tv_06-29-2022_NKEPn7mqmpk_0_1_4.mp3'
#audio_array,sr = librosa.load(filepath, sr=16000)

In [184]:
ipd.Audio(audio_array, autoplay=True, rate=16000)

# Pseudolabel the audio files

In [51]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor,Wav2Vec2ProcessorWithLM, AutoProcessor
from datasets import load_metric

metric = load_metric("wer")

ASR_SAMPLING_RATE = 16_000
MODEL_ID = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

lang_code="swh"


<ipython-input-51-5fb34a7c5336>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("wer")


In [200]:
cer_metric = load_metric("cer")

In [52]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [195]:
model.to(device)

processor.tokenizer.set_target_lang(lang_code)
model.load_adapter(lang_code)

In [223]:
def confidence_score(pred_scores, word_dict, index):
    probs = pred_scores[index, word_dict["start_offset"]: word_dict["end_offset"]]
    return round(torch.sum(probs).item() / (len(probs)), 5)


def transcribe(audio_samples, reference_sentence):
    transcription=""
    word_confidence_scores= {}
    cer = None
    wer = None
    try:
        audio_samples = audio_array
        #audio_samples = audio_sb_on_vf

        inputs = processor(
            audio_samples, sampling_rate=ASR_SAMPLING_RATE, return_tensors="pt", padding=True
        )

        with torch.no_grad():
            outputs = model(**inputs.to(device)).logits
            #outputs = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

        ids = torch.argmax(outputs, dim=-1)
        text = processor.decode(ids[0])
        transcription = text.lower()

        scores = torch.nn.functional.log_softmax(outputs, dim=-1)
        pred_scores = scores.gather(1, ids.unsqueeze(-1))[:, :, 0]

        output_with_offset = processor.batch_decode(ids, output_word_offsets=True)

        word_confidence_scores = {d["word"]: confidence_score(pred_scores, d, 0) for d in output_with_offset.word_offsets[0]}
        min_confidence = min(word_confidence_scores.values())

        values = [i for i in word_confidence_scores.values()]
        avg_confidence = round(sum(values)/len(values), 5)

        predicted_text = remove_special_characters(transcription)

        wer = metric.compute(predictions=[predicted_text], references=[labels])
        cer = cer_metric.compute(predictions=[predicted_text], references=[labels])

    except:
      transcription=""

    return transcription, word_confidence_scores,min_confidence,avg_confidence,wer,cer

In [224]:
transcription, word_confidence_scores,min_confidence,avg_confidence,wer,cer = transcribe(audio_sb_on_vf, labels)

In [225]:
def get_confidence_scores_helper(batch, target_audio,labels, prefix):
  try:
    transcription, word_confidence_scores,min_confidence,avg_confidence,wer,cer = transcribe(target_audio, labels)

    batch[prefix+"_transcription"] = transcription
    batch[prefix+"_confidence_scores"] = word_confidence_scores
    batch[prefix+"_min_confidence_scores"] = min_confidence
    batch[prefix+"_avg_confidence_scores"] = avg_confidence
    batch[prefix+"_wer"] = wer
    batch[prefix+"_cer"] = cer
  except:
    batch[prefix+"_transcription"] = None
    batch[prefix+"_confidence_scores"] = None
    batch[prefix+"_min_confidence_scores"] = None
    batch[prefix+"_avg_confidence_scores"] = None
    batch[prefix+"_wer"] = None
    batch[prefix+"_cer"] = None

  return batch

def get_confidence_scores_batch(batch):
  audio_array = batch['audio']["array"]
  audio_vf = batch["audio_vf"]["array"]
  audio_sb = batch["audio_sb"]["array"]
  audio_sb_on_vf = batch["audio_sb_on_vf"]["array"]

  labels = remove_special_characters(batch["sentence"])
  try:
    get_confidence_scores_helper(batch, audio_array,labels, "audio")
    get_confidence_scores_helper(batch, audio_vf,labels, "audio_vf")
    get_confidence_scores_helper(batch, audio_sb,labels, "audio_sb")
    get_confidence_scores_helper(batch, audio_sb_on_vf,labels, "audio_sb_on_vf")
  except:
    print("Error during processing")
  return batch

In [216]:
scored_dataset = split_dataset.map(
    get_confidence_scores_batch
)

'kuinathifisha meli iliweze kuendelea kufanya kazi kwa miaka mingi zaidi'

In [ ]:
scored_dataset.push_to_hub(scored_dataset_name, private=True)